In [1]:
from decouple import config
import pandas as pd
import sys
import os
import unicodedata
sys.path.insert(0, config('PROYECTO_DIR'))
from clases.bd.conexion import  SQLServerConnector
conn = SQLServerConnector('DBGERESA')
pd.set_option('display.max_columns', None)



✅ Conexión exitosa a la base de datos 'DBGERESA'


In [2]:
def limpiar_columna(texto):
    if pd.isna(texto):
        return texto
    # Quitar acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    # Reemplazos
    return texto.replace(' ', '_').replace('.', '')

# Ruta de la carpeta
folder_path = os.path.join(config('PROYECTO_DATA'), "TAR-VIH/")

# Lista para almacenar todos los DataFrames
dataframes = []

# Iterar sobre todos los archivos .xlsx en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):        
        file_path = os.path.join(folder_path, filename)
        
        try:
            df = pd.read_excel(
                file_path,   # aquí antes ponías "archivo"
                skiprows=8,
                usecols="A:BK"
            )
            df.columns = [limpiar_columna(col) for col in df.columns]

            if filename.startswith("trama_recibida"):
                print("Archivo encontrado:", filename)
            
            dataframes.append(df)

        except Exception as e:
            print(f"Error al leer {filename}: {e}")

# Unir todos los DataFrames en uno solo
final_df = pd.concat(dataframes, ignore_index=True)

# Mostrar primeras 3 filas
final_df.head(3)
conn.insertar_dataframe(final_df, 'CG10_TAR_VIH', if_exists='replace')

C:\Users\Irvin\AppData\Local\Temp\ipykernel_13116\32270879.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)
C:\Users\Irvin\AppData\Local\Temp\ipykernel_13116\32270879.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)
C:\Users\Irvin\AppData\Local\Temp\ipykernel_13116\32270879.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a fu

✅ DataFrame insertado correctamente en 'CG10_TAR_VIH' en lotes de 100000 registros.
